Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

## S20. Buscar la distancia mínima que existe entre dos locales de McDonald ‘s en el estado de LA (que no comparten dirección). ¿Cuál es la distancia?¿A qué ciudad/es pertenecen? ¿Cuáles son las direcciones de los locales?

### Installs e imports

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=982f30c407f8b4bbed0f15feed04385f25414446b3b5366103b8959e565dccc3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After t

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
import numpy as np

### Resolución

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('/content/drive/MyDrive/Datos/TP2/business_parquet', inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Me quedo con los locales de McDonald's en el estado de LA, con su latitud y londitud, sus ciudades y sus direcciones:

In [ ]:
filtered = rdd.filter(lambda x: x.name == "McDonald's" and x.state == 'LA').map(lambda x: (1, ((x.latitude, x.longitude), x.city, x.address)) ).cache()
filtered.take(5)

[(1, ((30.0042438706, -90.0364068849), 'New Orleans', '4240 Louisa St')),
 (1, ((29.9091675264, -90.2043836564), 'Avondale', '2951 Hwy 90')),
 (1,
  ((30.0068142619, -90.2007519454),
   'Metairie',
   '5315 Veterans Memorial Blvd')),
 (1, ((29.963288674, -90.1118321), 'New Orleans', '3443 S Carrollton')),
 (1, ((30.0012184272, -90.1260017558), 'Metairie', '301 Veterans Hwy'))]

Calculo la distancia entre todos los pares de locales cuyas direcciones no coinciden, quedandome con la distancia, las ciudades y las direcciones:

In [ ]:
def distance(x, y):
  return np.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2)

distances = filtered.join(filtered).filter(lambda x: x[1][0][2] != x[1][1][2])\
  .map(lambda x: (distance(x[1][0][0], x[1][1][0]), (x[1][0][1], x[1][1][1]), (x[1][0][2], x[1][1][2])))

Encuentro los locales con menor distancia entre si:

In [ ]:
distances.reduce(lambda x, y: x if x[0] < y[0] else y)

(0.011741207401921507,
 ('Gretna', 'Gretna'),
 ('1212 Stumpf Blvd', '501 Westbank Expy'))

Los locales de McDonald's del estado de LA más cercanos están a distancia 0.012 aproximadamente, pertenecen a la ciudad de Gretna y sus direcciones son '1212 Stumpf Blvd' y '501 Westbank Expy'.